In [1]:
# Import necessary libraries
import os
import json
from langchain.document_loaders import PyMuPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

In [2]:
# Load environment variables from .env file
load_dotenv()

True

## Task 1: Source Discovery

In [3]:
# 1) Find all relevant sources about yourself

# List your personal documents
personal_docs = [
    "../data/linkedin_profile.pdf",
    "../data/personal_bio.txt",
    "../data/AIT_SIS_personal_info.txt"
]

# Load documents
documents = []
for doc_path in personal_docs:
    try:
        if doc_path.endswith('.pdf'):
            loader = PyMuPDFLoader(doc_path)
        elif doc_path.endswith('.txt'):
            loader = TextLoader(doc_path)
        else:
            print(f"Unsupported file type: {doc_path}")
            continue
        docs = loader.load()
        # Add source metadata to each document
        for doc in docs:
            doc.metadata["source"] = os.path.basename(doc_path)
        documents.extend(docs)
        print(f"Loaded: {doc_path}")
    except Exception as e:
        print(f"Error loading {doc_path}: {e}")

print(f"Total documents loaded: {len(documents)}")

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200
)
doc_chunks = text_splitter.split_documents(documents)
print(f"Created {len(doc_chunks)} document chunks")

Loaded: ../data/linkedin_profile.pdf
Loaded: ../data/personal_bio.txt
Loaded: ../data/AIT_SIS_personal_info.txt
Total documents loaded: 3
Created 3 document chunks


In [4]:
# Set up embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create vector store
vectorstore = FAISS.from_documents(doc_chunks, embedding_model)

# Save vectorstore locally
vectorstore.save_local("vectorstore")
print("Vector store saved successfully")

C:\Users\Mir Ali\AppData\Local\Temp\ipykernel_25980\1701810785.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



Vector store saved successfully


c:\Users\Mir Ali\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [5]:
template = """
You are a helpful assistant that answers questions about Mir Ali Use the following context and chat history to provide a gentle and informative response. If the context doesn't provide the answer, politely say you don't have enough information.

Context: {context}

Chat History: {chat_history}

Question: {question}

Answer:
"""

PROMPT = PromptTemplate(
    input_variables=["context", "chat_history", "question"],
    template=template
)

In [6]:
groq_api_key = os.getenv("GROQ_API_KEY")
if not groq_api_key:
    print("Error: GROQ_API_KEY environment variable not set.")
    exit()

groq_llm = ChatGroq(
    api_key=groq_api_key,
    model_name="llama-3.3-70b-versatile"
)

In [7]:
# Set up conversation memory
memory = ConversationBufferWindowMemory(
    k=5,
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

C:\Users\Mir Ali\AppData\Local\Temp\ipykernel_25980\2504715149.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


In [8]:
# Create RAG chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=groq_llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    memory=memory,
    return_source_documents=True,
    combine_docs_chain_kwargs={"prompt": PROMPT}
)

In [9]:
# List retriever and generator models
print("Retriever Model: FAISS with HuggingFace embeddings (sentence-transformers/all-MiniLM-L6-v2)")
print("Generator Model: Groq's llama-3.3-70b-versatile")

# 5) Analyze potential issues
print("""
Analysis of Issues:
- Retriever: FAISS may retrieve irrelevant chunks if embeddings fail to capture semantic meaning accurately. This could happen with ambiguous questions or insufficient document detail.
- Generator: Groq's Llama3-70b might generate plausible but incorrect answers (hallucination) if retrieved context is incomplete.
- Mitigation: Ensure documents are comprehensive, adjust chunk size/overlap, or refine the prompt to prioritize context adherence.
""")

# Task 3: Chatbot Development

# Define the 10 required questions
questions = [
    "How old are you?",
    "What is your highest level of education?",
    "What major or field of study did you pursue during your education?",
    "How many years of work experience do you have?",
    "What type of work or industry have you been involved in?",
    "Can you describe your current role or job responsibilities?",
    "What are your core beliefs regarding the role of technology in shaping society?",
    "How do you think cultural values should influence technological advancements?",
    "As a master's student, what is the most challenging aspect of your studies so far?",
    "What specific research interests or academic goals do you hope to achieve during your time as a master's student?"
]

# Generate answers and store in JSON format
results = []
for i, question in enumerate(questions, start=1):
    response = qa_chain({"question": question})
    answer = response["answer"]
    results.append({
        "question_number": i,
        "question": question,
        "answer": answer
    })
    # Print the question number, question, and answer
    print(f"Question {i}: {question}")
    print(f"Answer: {answer}\n")

# Save the results to a JSON file
with open("answers.json", "w") as f:
    json.dump(results, f, indent=2)
print("Answers saved to 'answers.json'")

C:\Users\Mir Ali\AppData\Local\Temp\ipykernel_25980\168687936.py:32: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({"question": question})


Retriever Model: FAISS with HuggingFace embeddings (sentence-transformers/all-MiniLM-L6-v2)
Generator Model: Groq's llama-3.3-70b-versatile

Analysis of Issues:
- Retriever: FAISS may retrieve irrelevant chunks if embeddings fail to capture semantic meaning accurately. This could happen with ambiguous questions or insufficient document detail.
- Generator: Groq's Llama3-70b might generate plausible but incorrect answers (hallucination) if retrieved context is incomplete.
- Mitigation: Ensure documents are comprehensive, adjust chunk size/overlap, or refine the prompt to prioritize context adherence.

Question 1: How old are you?
Answer: To determine your age, I'll need to calculate the difference between the current year and your birth year. Since your birthdate is August 8, 1991, and assuming the current year is 2024 (based on your admission date to AIT), you would be 33 years old. However, please note that I don't have real-time information, so if the current year is different, your 